# Optimization Models

**In this notebook, we explore a linear programming problem using the pulp libeary in python.**

The problem requires us to minimize cost by: 
- Selecting the optimum size and location for opening plants. 
- Selecting the optimum number of units to supply to each of the the markets.

While making sure that:
- The capacity of plants at each of the locations are not exceeded.
- The demand for each of the markets are fulfilled.

## Imporitng Libraries and Datasets 

In [1]:
from pulp import *
import pandas as pd
import numpy as np

In [2]:
#Manufacturing cost per unit
man_costs = pd.read_excel("data/variable costs.xlsx", index_col=0)
man_costs

,USA,GERMANY,JAPAN,BRAZIL,INDIA
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
GERMANY,13,13,13,13,13
JAPAN,10,10,10,10,10
BRAZIL,8,8,8,8,8
INDIA,5,5,5,5,5


In [3]:
#Freight cost per unit
freight_costs = pd.read_excel("data/freight costs.xlsx", index_col=0)
freight_costs = freight_costs
freight_costs

,USA,GERMANY,JAPAN,BRAZIL,INDIA
Freight Costs ($/Container),,,,,
USA,0,1750,1100,2300,1254
GERMANY,1905,0,1231,2892,1439
JAPAN,2200,3250,0,6230,2050
BRAZIL,2350,3150,4000,0,4250
INDIA,1950,2200,3500,4200,0


In [4]:
#Total variable cost per unit
var_costs = man_costs+freight_costs/1000
var_costs

,USA,GERMANY,JAPAN,BRAZIL,INDIA
Variable Costs ($/Unit),,,,,
USA,12.000,13.75,13.100,14.300,13.254
GERMANY,14.905,13.00,14.231,15.892,14.439
JAPAN,12.200,13.25,10.000,16.230,12.050
BRAZIL,10.350,11.15,12.000,8.000,12.250
INDIA,6.950,7.20,8.500,9.200,5.000


In [10]:
#Total fixed cost per 1000 unit
fixed_costs = pd.read_excel("data/fixed cost.xlsx", index_col=0)
fixed_costs

,LOW,HIGH
USA,6500,9500
GERMANY,4980,7270
JAPAN,6230,9100
BRAZIL,3230,4730
INDIA,2110,6160


In [6]:
#Plant capacities in 1000 units
capacities = pd.read_excel("data/capacity.xlsx", index_col=0)
capacities

,LOW,HIGH
Capacity (kUnits/month),,
USA,500,1500
GERMANY,500,1500
JAPAN,500,1500
BRAZIL,500,1500
INDIA,500,1500


In [7]:
#Market demands
demands = pd.read_excel("data/demand.xlsx", index_col=0)
demands

,Demand
(Units/month),
USA,2800000
GERMANY,90000
JAPAN,1700000
BRAZIL,145000
INDIA,160000


## Defining Decision Variables

In [8]:
locations = ["USA", "GERMANY", "JAPAN", "BRAZIL", "INDIA"]
sizes = ["LOW", "HIGH"]

plant_to_size = [(i,s) for s in sizes for i in locations]
plant_to_plant = [(i,j) for i in locations for j in locations]

In pulp decision variable indices are defined as dictionaries. Take the plant_to_size dictionary for example

In [9]:
plant_to_size

[('USA', 'LOW'),
 ('GERMANY', 'LOW'),
 ('JAPAN', 'LOW'),
 ('BRAZIL', 'LOW'),
 ('INDIA', 'LOW'),
 ('USA', 'HIGH'),
 ('GERMANY', 'HIGH'),
 ('JAPAN', 'HIGH'),
 ('BRAZIL', 'HIGH'),
 ('INDIA', 'HIGH')]

## Model Definition 

In [55]:
#Model Definition
model = LpProblem("Production plant location & supply optimization", LpMinimize)

C:\Users\vubanc\AppData\Local\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


After the model has been defined, we will add decision variables, objective functions and variables, one by one. As of now, the model formulation contains none of these as we can see below:

In [56]:
model

Production_plant_location_&_supply_optimization:
MINIMIZE
None
VARIABLES

## Creating Decision Variables 

In [57]:
x = LpVariable.dicts("plant_open", plant_to_size, cat = "Binary")
y = LpVariable.dicts("units_supplied", plant_to_plant,  lowBound=0, upBound=None, cat="Continuous")

The decision variables are dictionaries, where the keys are indices and the values are the optimum values which will be assigned once the model is solved. Right now, the values only have placeholders, which will be replaced by optimum values.

In [58]:
x

{('USA', 'LOW'): plant_open_('USA',_'LOW'),
 ('GERMANY', 'LOW'): plant_open_('GERMANY',_'LOW'),
 ('JAPAN', 'LOW'): plant_open_('JAPAN',_'LOW'),
 ('BRAZIL', 'LOW'): plant_open_('BRAZIL',_'LOW'),
 ('INDIA', 'LOW'): plant_open_('INDIA',_'LOW'),
 ('USA', 'HIGH'): plant_open_('USA',_'HIGH'),
 ('GERMANY', 'HIGH'): plant_open_('GERMANY',_'HIGH'),
 ('JAPAN', 'HIGH'): plant_open_('JAPAN',_'HIGH'),
 ('BRAZIL', 'HIGH'): plant_open_('BRAZIL',_'HIGH'),
 ('INDIA', 'HIGH'): plant_open_('INDIA',_'HIGH')}

##  Defining the Objective Function

In [59]:
model += (lpSum([x[(i,s)]*fixed_costs.loc[i,s]*1000 for s in sizes for i in locations]) + 
          lpSum([y[(i,j)]*var_costs.loc[i,j] for i in locations for j in locations]))

## Defining the Constraints

In [60]:
for j in locations:
    model += lpSum([y[(i,j)] for i in locations]) == demands.loc[j,"Demand"]
    
for i in locations:
    model += lpSum([y[(i,j)] for j in locations]) <= lpSum([capacities.loc[i,s]*x[(i,s)]*1000 for s in sizes])

## Solving the Model

Printing the LpStatus shows that the we found the optimal solution for the model.

In [61]:
model.solve()
print(LpStatus[model.status])

Optimal


## Optimal Solutions 

To view the optimal solutions, fiest we create dataframes for each of the decision variables:

In [62]:
x_df = pd.DataFrame(data=[x[plant_to_size[i]].varValue for i in range (len(plant_to_size))], index=[i + "_" + s for s in sizes for i in locations], columns=["Plant Opening"])
y_df = pd.DataFrame(data=[y[(plant_to_plant[i])].varValue for i in range(len(plant_to_plant))], index = [i + "_" + j for i in locations for j in locations], columns = ["Amount Supplied"])

In [63]:
print(f"Optimal Cost: ${int(value(model.objective))}")

Optimal Cost: $65118000


In [64]:
x_df

,Plant Opening
USA_LOW,0.0
GERMANY_LOW,0.0
JAPAN_LOW,0.0
BRAZIL_LOW,0.0
INDIA_LOW,1.0
USA_HIGH,0.0
GERMANY_HIGH,0.0
JAPAN_HIGH,1.0
BRAZIL_HIGH,1.0
INDIA_HIGH,1.0


In [65]:
y_df

,Amount Supplied
USA_USA,0.0
USA_GERMANY,0.0
USA_JAPAN,0.0
USA_BRAZIL,0.0
USA_INDIA,0.0
GERMANY_USA,0.0
GERMANY_GERMANY,0.0
GERMANY_JAPAN,0.0
GERMANY_BRAZIL,0.0
GERMANY_INDIA,0.0
